In [1]:
!wget https://dcc.ligo.org/public/0171/P2000434/001/Multiple-Fig-Data.tar.gz

--2021-04-13 19:25:47--  https://dcc.ligo.org/public/0171/P2000434/001/Multiple-Fig-Data.tar.gz
Resolving dcc.ligo.org (dcc.ligo.org)... 131.215.125.144
Connecting to dcc.ligo.org (dcc.ligo.org)|131.215.125.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1155640816 (1.1G) [application/x-gzip]
Saving to: ‘Multiple-Fig-Data.tar.gz’

100%[====================================>] 1,155,640,816 52.2MB/s   in 19s    

2021-04-13 19:26:06 (58.6 MB/s) - ‘Multiple-Fig-Data.tar.gz’ saved [1155640816/1155640816]



In [2]:
!tar -xvf Multiple-Fig-Data.tar.gz

Multiple-Fig-Data/
Multiple-Fig-Data/.DS_Store
Multiple-Fig-Data/effective_spin_ppd/
Multiple-Fig-Data/effective_spin_ppd/default-spin-xeff-xp-ppd-data.json
Multiple-Fig-Data/effective_spin_ppd/gaussian-spin-xeff-xp-ppcheck-data.json
Multiple-Fig-Data/effective_spin_ppd/gaussian-spin-xeff-xp-ppd-data.json
Multiple-Fig-Data/get_mass_ppd_statistics.py
Multiple-Fig-Data/mass_ppd/
Multiple-Fig-Data/mass_ppd/.DS_Store
Multiple-Fig-Data/mass_ppd/all_events/
Multiple-Fig-Data/mass_ppd/all_events/.gitattributes
Multiple-Fig-Data/mass_ppd/all_events/o1o2o3_mass_b_iid_mag_two_comp_iid_tilt_powerlaw_redshift_mass_data.h5
Multiple-Fig-Data/mass_ppd/all_events/o1o2o3_mass_c_iid_mag_two_comp_iid_tilt_powerlaw_redshift_mass_data.h5
Multiple-Fig-Data/mass_ppd/all_events/o1o2o3_mass_d_iid_mag_two_comp_iid_tilt_powerlaw_redshift_mass_data.h5
Multiple-Fig-Data/mass_ppd/all_events/o1o2o3_mass_e_iid_mag_two_comp_iid_tilt_powerlaw_redshift_mass_data.h5
Multiple-Fig-Data/mass_ppd/default/
Multiple-Fig-Data/m

In [ ]:
parameter_translator = dict(
    mass_1 = 'mass_1_source',
    mass_2 = 'mass_2_source',
    mass_ratio = 'mass_ratio',
    luminosity_distance = 'luminosity_distance',
    a_1 = 'a_1',
    a_2 = 'a_2',
    cos_tilt_1 = 'cos_tilt_1',
    cos_tilt_2 = 'cos_tilt_2', 
    chi_eff = 'chi_eff', 
    chi_p = 'chi_p', 
    redshift = 'redshift')

j = 0

for file in glob.glob("GWTC-2/GWTC-2_sample_release/S*.h5"):
    print(i, file)
    i = i+1
    with h5py.File(file, 'r') as ff:
        for key in ["C01:NRSur7dq4", "C01:IMRPhenomPv3HM", "C01:IMRPhenomPv2"]:
            print(j)
            j = j+1
            if key in ff.keys():
                _posterior = pd.DataFrame()
                for my_key, gwtc_key in parameter_translator.items():
                    post = ff[key]["posterior_samples"]
                    f = pd.DataFrame(np.array(post))
                    _posterior[my_key] = f[gwtc_key]


            nSamples = _posterior["chi_eff"].size

            joint_priors = np.zeros(nSamples)
            chi_eff_priors = np.zeros(nSamples)
            chi_p_priors = np.zeros(nSamples)

            for m in range(nSamples):
                ref_joint_prior_points = [evaluate_joint_prior(_posterior["chi_eff"][m], _posterior["chi_p"][m], spline) for spline in joint_prior_splines]
                joint = UnivariateSpline(q_grid, ref_joint_prior_points, k = 3, s = 0.1)
                joint_priors[m] = joint(_posterior["mass_ratio"][m])

                ref_chieff_prior_points = [interp(_posterior["chi_eff"][m]) for interp in chi_eff_prior_interps]
                chieff_1d = UnivariateSpline(q_grid, ref_chieff_prior_points, k = 3, s = 0.1)
                chi_eff_priors[m] = chieff_1d(_posterior["mass_ratio"][m])

                ref_chip_prior_points = [interp(_posterior["chi_p"][m]) for interp in chi_p_prior_interps]
                chip_1d = UnivariateSpline(q_grid, ref_chip_prior_points, k = 3, s = 0.1)
                chi_p_priors[m] = chip_1d(_posterior["mass_ratio"][m])

            # Mass priors

            prior_astro = astro_prior(_posterior["mass_1"],_posterior["mass_2"], _posterior["redshift"], _posterior["luminosity_distance"])
            prior_astro[prior_astro < 0] = 0
            prior_LAL = LAL_prior(_posterior["redshift"], _posterior["luminosity_distance"])
            weights = prior_astro/prior_LAL

            _posterior["joint_prior"] = joint_priors
            _posterior["chi_eff_prior"] = chi_eff_priors
            _posterior["chi_p_prior"] = chi_p_priors
            _posterior["weights"] = weights

            posteriors.append(_posterior)

            filename = re.split('\/|\.',file)[-2]
            _posterior.to_csv("./preprocessed_events/posteriors_with_prior/{}.csv".format(filename), index = False) 
        
            break;

In [ ]:
filename = re.split('\/|\.',file)[-2]
_posterior.to_csv("../processed_samples/OverallPosteriors/{}.csv".format(filename), index = False) 

In [ ]:
# Check the structure of VT file

vt_file = '../Sensitivity/O3aSensitivity/o3a_bbhpop_inj_info.hdf'

ff = h5py.File(vt_file, 'r')
print(list(ff.keys()))
print(list(ff['injections'].keys()))
print(list(ff.attrs.items()))